<a href="https://colab.research.google.com/github/PDBeurope/pdbe-notebooks/blob/main/variants_embl-ebi_may2024/structures_available.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**<H1>Structures available</H1>**
**Using PDBe-KB & 3D-Beacons to gain understanding on genetic variants**
<img src="https://www.ebi.ac.uk/pdbe/docs_dev/logos/images/RGB/PDBe-logo-RGB_2013.png" height="300" align="right">

#Welcome to this notebook!

To use this notebook you can:

(1) Click on link at top of the page when viewing the file in Github

*   you will need to have a Google account
*   be logged in to Google Colab <br>(by being logged into Google account)

<br>

(2) Visit the Colaboratory page:<br>
https://colab.research.google.com/<br>
and access the following Github repository *via* the interface:<br>
https://github.com/PDBeurope/pdbe-notebooks/

<br>

(3) Visit the Colaboratory page:<br>
https://colab.research.google.com/<br>
and upload the file from the Colaboratory interface

*   you will need to have a Google account
*   be logged in to Google Colab <br>(by being logged into Google account)



---

  ## How to use this notebook:
1. To run a code cell, click on the cell to select it. You will notice a play button (▶️) on the left side of the cell. Click on the play button or **press Shift+Enter** to run the code in the selected cell.
2. The code will start executing, and you will see the output, if any, displayed below the code cell.
3. Move to the next code cell and repeat steps 2 and 3 until you have executed all the desired code cells in sequence.
4. The currently running step is indicated by a circle with a stop sign next to it.
If you need to stop or interrupt the execution of a code cell, you can click on the stop button (■) located next to the play button.

*Remember to run the code cells in the correct order, as their execution might depend on variables or functions defined in previous cells. You can modify the code in a code cell and re-run it to see updated results.*

<br>

---

## Contact us

If you experience any bugs please contact pdbehelp@ebi.ac.uk and put "Help with" and the title of the notebook in the subject line of the message.


---
# Why use this notebook:

<img src="https://github.com/glevans/7ADD-workshop-2024/blob/main/Images/API_image.png?raw=true" height="300" align="right">

This interactive Python notebook demonstrates accessing **Protein Data Bank**,

using some of the tools developed by the **Protein Data Bank in Europe (PDBe)**:

<br>

- [PDBe Knowledge Base -> pdbe-kb.org](https://www.ebi.ac.uk/pdbe/pdbe-kb/)

- [3D Beacons -> 3d-beacons.org](https://www.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/)

<br>

<p>An API (Application Programming Interface) is a programmatic way to obtain information.</p>

<p>Virtually every webpage you visit is using one or more APIs.</p>

<p>We will be tapping into the potential of APIs to more quickly access and assess information.</p>

<p> </p>

---
*We will briefly cover:*
### **3D-Beacons**

The main purpose of 3D-Beacons is to provide programmatic access to **BOTH** experimentally determined and theoretical structures from one web location.

Some example theoretical model providers:
* SWISS-MODEL
* AlphaFold DB
* AlphaFill
* ModelArchive
* isoform.io
* levylab

A more comprehensive list is available here: [Data providers](https://www.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/guidelines)

For direct link to 3D-Beacons API interface, visit [3D-Beacons API page](https://www.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/api/)

A link to main webpage:
[3D Beacons -> 3d-beacons.org](https://www.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/)
<br>

---
*We will focus more on:*
### **Protein Data Bank in Europe Knowledge Base (PDBe-KB)**

**PDBe-KB** provides more information and analysis specifically for experimentally-determined structures.

**3D Beacons** is being used <i>'behind-the-scenes'</i> on the **PDBe-KB** pages to enable theoretical macromolecule models to be displayed alongside experimental models.

**PDBe-KB** provides information on 3D macromolecular structures in the context of other macromolecular structures.

With **PDBe-KB** webpages and its APIs, the following information can be accessed:

* Identify PDB ids (aka multiple structures) known to contain related data due to one or more protein chains being associated with the same UniProt ID.
* Insight on compounds & residues from considering related structures in the context with each other.
* Mappings to CATH, EC, GO, InterPro, Pfam, SCOP, etc
* and more...

For direct link to PDBe-KB's Aggregated API, visit [PDBe-KB Aggregated API page](https://www.ebi.ac.uk/pdbe/graph-api/pdbe_doc/)

A link to main webpage:
[PDBe Knowledge Base -> pdbe-kb.org](https://www.ebi.ac.uk/pdbe/pdbe-kb/)
<br>


# 1)  Setting up

In [ ]:
#@title Press shift+enter to start

# @markdown <br>
# @markdown This section installs libraries and sets up Python code to extract information from PDBe-KB's APIs.
# @markdown <br>
# @markdown <br>
# @markdown After running this cell, this Notebook will be able to connect 3D-Beacons and PDBe-KB's APIs.
# @markdown <br>
# @markdown It will create summary tables for some information about the structures associated with a UniProt ID.
# @markdown <br>
# @markdown <br>
# @markdown The information listed below can also be found either on PDBe-KB pages and/or utilizing 3D Beacons.
# @markdown <br>


######## LIBRARIES

import requests, sys, json
import pandas as pd
from pprint import pprint

print("Succesfully installed!")

####### 3D BEACONS FUNCTIONS
#defining functions for search and summary

def _search_3DBeacons(UniProt_ID):
    try:
        api_url = f"https://www.ebi.ac.uk/pdbe/pdbe-kb/3dbeacons/api/v2/uniprot/summary/{UniProt_ID}.json"
        response = requests.get(api_url)
        response.raise_for_status()
        return response.json().get("structures", [])
    except requests.exceptions.RequestException as err:
        print("Double check if the UniProt id is correct.")
        print()
        raise Exception(f"Error fetching data: {err}")

def _create_structure_dataframe(structures):
    if structures:
        details_list = [
            [
                s["summary"]["model_identifier"],
                s["summary"]["model_category"],
                s["summary"]["provider"],
                s["summary"]["uniprot_start"],
                s["summary"]["uniprot_end"],
            ]
            for s in structures
        ]
        columns = ["ID", "Type", "Provider", "UniProt start", "UniProt end"]
        df = pd.DataFrame(details_list, columns=columns)
        return df
    else:
        raise Exception("No structures found for the given UniProt ID.")

def all_structure_data(UniProt_ID):
    try:
        structures = _search_3DBeacons(UniProt_ID)
        return _create_structure_dataframe(structures)
    except Exception as e:
        return f"Error: {e}"

def displaying_3dbeacon_summary(UniProt_ID):
    df_3D_beacons = all_structure_data(UniProt_ID)
    if type(df_3D_beacons) == str:
        print("Something went wrong. Check the UniProt ID.")
    elif df_3D_beacons is not None:
        num_rows = df_3D_beacons.shape[0]
        print(f"There are {num_rows} structures available associated with UniProt ID {UniProt_ID}")
        print()
        return df_3D_beacons.head(num_rows)
    else:
        print("Something went wrong. Check the UniProt ID.")

def displaying_3dbeacon_exp_summary(UniProt_ID):
    df_3D_beacons = all_structure_data(UniProt_ID)
    if type(df_3D_beacons) == str:
        print("Something went wrong. Check the UniProt ID.")
    elif df_3D_beacons is not None:
        df_3D_beacons2 = df_3D_beacons.drop(df_3D_beacons[df_3D_beacons['Type'] == 'TEMPLATE-BASED'].index)
        df_3D_beacons3 = df_3D_beacons2.drop(df_3D_beacons[df_3D_beacons['Type'] == 'CONFORMATIONAL ENSEMBLE'].index)
        df_3D_beacons4 = df_3D_beacons3.drop(df_3D_beacons[df_3D_beacons['Type'] == 'AB-INITIO'].index)
        num_rows = df_3D_beacons4.shape[0]
        print(f"There are {num_rows} different structures, where experimental data was used for the structure, associated with UniProt ID {UniProt_ID}")
        print()
        return df_3D_beacons4.head(num_rows)
    else:
        print("Something went wrong. Check the UniProt ID.")

####### 3D BEACONS FUNCTIONS
#defining functions for reporting urls

def links_for_structures(UniProt_ID):
    try:
        structures = _search_3DBeacons(UniProt_ID)
        print(f"There are {len(structures)} listed associated with UniProt id {UniProt_ID}")
        list_of_lists =[]
        for index in range(0, len(structures)):
            structure_details = structures[index]['summary']
            model_identifier = structure_details['model_identifier']
            model_catergory = structure_details['model_category']
            provider = structure_details['provider']
            model_url = structure_details['model_url']
            model_format = structure_details['model_format']
            label = f"Click on url to download the coordinate file - {model_format} format:"
            details_list = {model_identifier : [label, model_url]}
            print()
            pprint(details_list)
            print()
    except Exception as e:
        return f"Error: {e}"

def display_urls_for_structures(UniProt_ID):
    try:
        link_list = links_for_structures(UniProt_ID)
    except Exception as e:
        return f"An error occurred: {e}"

######## PDBe-KB FUNCTIONS
#defining functions for search and summary for variants

def api_structure_PDBeKB(UniProt_ID):
    try:
        requestURL = f"https://www.ebi.ac.uk/pdbe/graph-api/uniprot/unipdb/{UniProt_ID}"
        response = requests.get(requestURL)
        response.raise_for_status()
        return json.loads(response.text)
    except requests.exceptions.RequestException as err:
        print(f"Request Error: {err}")
        return None

def _parse_variant_data(structure_data, UniProt_ID):

    if type(structure_data) == dict:
        structure_details = structure_data[UniProt_ID]['data']
        amount = len(structure_details)
        UniProt_ID = UniProt_ID
        UniProt_summary = []
        UniProt_summary_list = []
        row_list = []

        for index in range(0, amount): # PDB ids

            pdb_id = structure_details[index]['accession']
            accession_item = structure_details[index]

            modified_residues = _modified_list(accession_item['residues'])
            mutated_residues = _mutated_list(accession_item['residues'])

            if modified_residues == None and mutated_residues == None:
                pass
            elif modified_residues == None and mutated_residues != None:
                row_list = []
                for index in range(0, len(mutated_residues)):
                    mutated_residues[index].update({"PDB_ID" : pdb_id})
                row_list.append(mutated_residues)
            elif modified_residues != None and mutated_residues == None:
                row_list = []
                for index in range(0, len(modified_residues)):
                    modified_residues[index].update({"PDB_ID" : pdb_id})
                row_list.append(modified_residues)
            elif modified_residues != None and mutated_residues != None:
                row_list = []
                for index in range(0, len(modified_residues)):
                    modified_residues[index].update({"PDB_ID" : pdb_id})
                row_list.append(modified_residues)
                for index in range(0, len(mutated_residues)):
                    mutated_residues[index].update({"PDB_ID" : pdb_id})
                row_list.append(mutated_residues)
            UniProt_summary_list.extend(row_list)

        for number in range(0,len(UniProt_summary_list)):
            UniProt_summary += UniProt_summary_list[number]
        return UniProt_summary

def _modified_list(residues):
    modified_list = []  # Initialize the list outside of the loop
    for residue_info in residues:
        if 'modification' in residue_info:
            modified = _modification_info(residue_info)
            modified_list.append(modified)  # Append the mutation to the list
    if modified_list == []:
        return None
    else:
        return modified_list  # Return the list

def _mutated_list(residues):
    mutated_list = []  # Initialize the list outside of the loop
    for residue_info in residues:
        if 'mutation' in residue_info:
            mutation = _mutation_info(residue_info)
            mutated_list.append(mutation)  # Append the mutation to the list
    if mutated_list == []:
        return None
    else:
        return mutated_list  # Return the list

def _mutation_info(residue_info):
    if residue_info['startIndex'] == residue_info['endIndex']:
        residue_position = residue_info['startIndex']
        UniProt_amino_acid = residue_info['startCode']
        coordinate_amino_acid = residue_info['pdbCode']
        change = residue_info['mutationType']
        mutated_residue = {'change': change, 'position' : residue_position, 'from_aa' : UniProt_amino_acid, 'to_aa' : coordinate_amino_acid}
        return mutated_residue

def _modification_info(residue_info):
    if residue_info['startIndex'] == residue_info['endIndex']:
        residue_position = residue_info['startIndex']
        UniProt_amino_acid = residue_info['startCode']
        coordinate_amino_acid = residue_info['pdbCode']
        change = "Modification"
        modified_residue = {'change': change, 'position' : residue_position, 'from_aa' : UniProt_amino_acid, 'to_aa' : coordinate_amino_acid}
        return modified_residue

def variant_summary(UniProt_ID):
    try:
        structure_data = api_structure_PDBeKB(UniProt_ID)
        UniProt_info = _parse_variant_data(structure_data, UniProt_ID)
        return UniProt_info
    except Exception as e:
        print("Double check if the UniProt id is correct.")
        print()
        return f"Error: {e}"

def displaying_variant_summary(UniProt_ID):
    list_of_dict = variant_summary(UniProt_ID)
    if type(list_of_dict) == str:
        print("Something went wrong. Check the UniProt ID.")
    elif list_of_dict== []:
        print("Something went wrong. Check the UniProt ID.")
    elif list_of_dict is not None:
        print(f"This is the output from entering the UniProt ID {UniProt_ID}.")
        df_variants = pd.DataFrame.from_dict(list_of_dict)
        df_variants2 = df_variants.sort_values(['position'], na_position='first', ascending=True, key=pd.to_numeric)
        df_variants3 = df_variants2.drop_duplicates() # Remove duplicate instance of same conflicts for same PDB ids
        df_variants4 = df_variants3.reset_index()
        df_variants5 = df_variants4.drop(['index'], axis=1)
        num_rows = df_variants5.shape[0]
        print()
        return df_variants5.head(num_rows)
    else:
        print("Something went wrong. Check the UniProt ID.")

######## PDBe-KB FUNCTIONS
#defining functions for search and summary

def parse_general_structure_data(structure_data, UniProt_ID):
    structure_details = structure_data[UniProt_ID]['data']

    accession_id_list = []
    UniProt_summary = {}
    print(f"There are {len(structure_details)} experimentally-determined structures associated with UniProt id {UniProt_ID}.")
    print()

    for number in range(0,len(structure_details)):
        accession_item = structure_details[number]
        accession_id = accession_item['accession']
        accession_id_list.append(accession_id)

        additionalData = accession_item['additionalData']
        resolution = additionalData['resolution']
        resolution = round(resolution,3)
        experiment = additionalData['experiment']

        if experiment == 'X-ray diffraction':
            concatenated_list = [experiment,resolution]
            UniProt_row = {accession_item['accession'] : concatenated_list}
        elif experiment == 'Electron Microscopy':
            concatenated_list = [experiment,resolution]
            UniProt_row = {accession_item['accession'] : concatenated_list}
        elif experiment == 'Solution NMR':
            concatenated_list = [experiment,None]
            UniProt_row = {accession_item['accession'] : concatenated_list}
        else:
            concatenated_list = [experiment,resolution]
            UniProt_row = {accession_item['accession'] : concatenated_list}
        UniProt_summary.update(UniProt_row)
    return UniProt_summary

def general_summary(UniProt_ID):
  try:
    structure_data = api_structure_PDBeKB(UniProt_ID)
    UniProt_info = parse_general_structure_data(structure_data, UniProt_ID)
    return(UniProt_info)
  except Exception as e:
    print("Double check if the UniProt id is correct.")
    print()
    return f"Error: {e}"

def display_exp_structure_summary(UniProt_ID):
    try:
        data = general_summary(UniProt_ID)
        if type(data) == str:
            print()
            print("Something has gone wrong. Double-check if the UniProt ID is okay.")
        else:
            df1 = pd.DataFrame.from_dict(data, orient="index", columns=['Experimental Method', 'Resolution (in Angstroms)'])
            num_rows = df1.shape[0]
            return df1.head(num_rows)
    except Exception as e:
        print(f"An error occurred: {e}")

def comma_pdb_id_list(UniProt_ID):
    try:
        data = general_summary(UniProt_ID)
        if type(data) == str:
            print()
            print("Something has gone wrong. Double-check if the UniProt ID is okay.")
        else:
            df1 = pd.DataFrame.from_dict(data, orient="index", columns=['Experimental Method', 'Resolution (in Angstroms)'])
            index_as_list = df1.index.tolist()
            print(', '.join(map(str, index_as_list)))
    except Exception as e:
        print(f"An error occurred: {e}")

######## PDBe-KB FUNCTION
#converting a json into a comma-delimited list

def getting_list_of_PDB_ids(load):
    PDB_ID_list = []
    for item in range (0, len(load)):
        PDBid = load[item]["PDB_ID"]
        PDB_ID_list.append(PDBid)
    str_PDB_ID_list = [str(item) for item in PDB_ID_list]
    delimiter_comma = ", "
    print(delimiter_comma.join(str_PDB_ID_list))

######## PDBe-KB FUNCTION
#generate comma-delimited list of nonmodified PDB IDs

def nonmodified_PDB_IDs(UniProt_ID):
    general = general_summary(UniProt_ID)
    variant = variant_summary(UniProt_ID)

    PDB_ID_list = []
    for item in range (0, len(variant)):
        PDBid = variant[item]["PDB_ID"]
        PDB_ID_list.append(PDBid)
    PDB_IDs_with_variant_list = list(dict.fromkeys(PDB_ID_list))
    all_PDB_ID = list(general.keys())

    s = set(PDB_IDs_with_variant_list)
    PDB_IDs_without_variant_list = [x for x in all_PDB_ID if x not in s]
    print(f"Of these there are {len(PDB_IDs_without_variant_list)} experimentally-determined structures without any modifications or discrepancies with UniProt id {UniProt_ID}.")

    print()

    PDB_ID_list = []
    for item in range (0, len(PDB_IDs_without_variant_list)):
        PDBid = PDB_IDs_without_variant_list[item]
        PDB_ID_list.append(PDBid)
    str_PDB_ID_list = [str(item) for item in PDB_ID_list]
    delimiter_comma = ", "
    print(delimiter_comma.join(str_PDB_ID_list))

######## PDBe-KB and 3D Beacon FUNCTION
#converting a json into a comma-delimited list

def diff_btwn_PDBeKB_3DBeacons(UniProt_ID):
    print("Output from PDBe-KB:")
    general = general_summary(UniProt_ID)
    all_PDB_IDs = list(general.keys())

    print("Output from 3D-Beacons for experimentally-determined structures:")
    df_all_expdata = displaying_3dbeacon_exp_summary(UniProt_ID)
    all_expdata_list = df_all_expdata ["ID"].tolist()

    s = set(all_expdata_list)
    PDB_ids_not_in_3Dbeacons = [x for x in all_PDB_IDs if x not in s]
    print(PDB_ids_not_in_3Dbeacons)

# 2)  Structures available using **3D-Beacons**
This section reports on predicted and experimentally-determined structures structures for a UniProt ID that are available *via* **3D-Beacons**.

---
These structures/models have a *'type'* assigned such as:

* EXPERIMENTALLY DETERMINED
* CONFORMATIONAL ENSEMBLE
* TEMPLATE-BASED
* AB-INITIO

If the *'Type'* is not 'EXPERIMENTALLY DETERMINED' or 'CONFORMATIONAL ENSEMBLE' then it is a predicted structure.

In [ ]:
#@title 2.1.  Get predicted structures, as well as experimentally determined structures associated with a UniProt ID
#@markdown  In this sub-section you will be able to retrieve BOTH predicted structures, as well as experimentally determined structures associated with an Uniprot accession ID.
#@markdown  <br>
#@markdown  <br>
#@markdown  With 3D-Beacons there are options to search based on sequence and Ensembl id as well.
#@markdown  <br>
#@markdown  <br>

#Run this code to display the widget
UniProt_ID = "" #@param {type:"string"}

#Run this code to perform a API call and load result into a DataFrame
displaying_3dbeacon_summary(UniProt_ID)


In [ ]:
#@title 2.2. Get experimentally-determined structural data associated with a UniProt ID
#@markdown  In this sub-section you will be able to retrieve experimentally determined data (including the wwPDB and a curated repository of small angle scattering data and models) associated with an Uniprot accession ID.
#@markdown  <br>
#@markdown  <br>
#@markdown  <br>


#Run this code to display the widget
UniProt_ID = "" #@param {type:"string"}

#Run this code to perform a API call and load result into a DataFrame
displaying_3dbeacon_exp_summary(UniProt_ID)

In [ ]:
#@title 2.3.  Get clickable links to download coordinates
#@markdown  In this sub-section you will be able to retrieve urls for predicted and experimentally-determined structures associated with a UniProt ID.

#Run this code to display the widget
UniProt_ID = "" #@param {type:"string"}

#Run this code to perform a API call and gets urls and display the urls
display_urls_for_structures(UniProt_ID)


#3)  Structures available using **PDBe-KB**

This section reports on the PDB ids for the experimentally-determined structures where at least one of the protein chains in the structure is associated with a UniProt ID.

The identity of these structures as well as more analysis of structures are available *via* **PDBe-KB**.

Structures of variants / engineered mutations of proteins are some times part of research studies. This section provides options to explores whether these exist and what they are.

---
*More details:*
<p>The experimentaly-determined structures/data available <i>via</i> <strong>PDBe-KB</strong> are associated with UniProt IDs are due to both manual and semi-automated curation.</p>
<p>That is a scientific curator is checking sequence alignments, with a feedback loop with the scientist(s) depositing the data confirming the right UniProt IDs are assigned before the data is made public.</p>
<p>This means that if there are structures where there was a lot of engineered mutations or where two or more proteins were engineered to make a new polymer chain to gain scientific insight -- these may not be as findable with certain sequence-based search approaches but will be found listed on <strong>PDBe-KB</strong> pages.</p>
<p>If the appropriate UniProt IDs are not available at the time of publishing, then after the structures are published, and there is a semi-automated process to assign unmapped structures to the appropriate UniProt ID. There is a synching process that means ultimately all structures associated with a UniProt ID end up listed on both <strong>PDBe-KB</strong> and UniProt pages</p>

In [ ]:
#@title 3.1. Get experimentally-determined coordinates associated with a UniProt ID
#@markdown  In this sub-section you will be able to retrieve experimentally-determined coordinates where based on sequence and reported taxonomy there is one or more chains associated with the same specified Uniprot accession ID.

#Run this code to display the widget
UniProt_ID = "" #@param {type:"string"}

#Run this code to perform a API call and load result into a DataFrame
display_exp_structure_summary(UniProt_ID)

In [ ]:
#@title 3.2. Press shift+enter to generate a list for copying-and-pasting
#@markdown Here is a copy-and-paste comma list of all the PDB ids for one UniProt id.
#@markdown <br>
#@markdown <br>

#Run this code to display the widget
UniProt_ID = "" #@param {type:"string"}

#Run this code to perform a API call and shows results as comma-delimited list
comma_pdb_id_list(UniProt_ID)

<img src="https://github.com/glevans/7ADD-workshop-2024/blob/main/Images/Mol_Viewer_Loading_Multiples.png?raw=true" align="right">

The above list can be pasted into the [Molstar viewer](https://molstar.org/viewer/) and upon *'Apply'* all the coordinates will be loaded into the viewer.

In [ ]:
#@title 3.3. Press shift+enter to generate a list for copying-and-pasting
#@markdown Here is a copy-and-paste comma list of the PDB ids with no modifications or discrepancy compared UniProt id.
#@markdown <br>
#@markdown <br>

#Run this code to display the widget
UniProt_ID = "" #@param {type:"string"}

#Run this code to perform a API call and shows results as comma-delimited list
nonmodified_PDB_IDs(UniProt_ID)

In [ ]:
#@title 3.4. Get summary of sequence discrepancy (*e.g.* variants) compared to the UniProt sequence
#@markdown In this sub-section the variants, engineered mutations, modified residues, etc for experimental structures will be reported.
#@markdown <br>
#@markdown <br>
#@markdown These have been identified as positions that are in conflict with the UniProt sequence.

#Run this code to display the widget
UniProt_ID = "" #@param {type:"string"}

#@markdown <br>
#@markdown Use filter button and imput the residue number to see if there is an experimental structure for a particular genetic variant.
#@markdown
#@markdown <br>
#@markdown <br>

displaying_variant_summary(UniProt_ID)

In [ ]:
#@title 3.5. Convert specific set of sequence discrepancy (*e.g.* variants) to a comma-delimited list
#@markdown In this sub-section the PDB ids with a specific variant / engineered mutation / modified residues / etc for experimental structures will be converted into a comma-delimited list.
#@markdown <br>
#@markdown <br>

#Run this code to display the widget
JSON = None #@param {type:"raw"}

getting_list_of_PDB_ids(JSON)

# 4) Comparing results from **3D Beacons** and **PDBe-KB**

On relatively rare occasions more experimentally-determined PDB structures will be listed associated with a UniProt id from a **PDBe-KB** API compared to a **3D Beacons** API.

The below does a comparison and reports the PDB ids listed from **PDBe-KB** but not from **3D-Beacons** for a UniProt id.

On the rare occasions that there are more PDB ids listed from **PDBe-KB** APIs compared to **3D Beacons** APIs, these proteins typically have multiple engineered mutations. Thus from a sequence point of view appear to be quite different from the wild-type protein, however by structure, can be overall similar with subtle differences.

It should be also noted that there are Cryo Electron Microscopy (CryoEM) maps, where coordinates/models are NOT built, due to limitations of resolution or other reasons. These are not findable using **PDBe-KB** or **3D Beacons** APIs. These Cryo EM maps where coordinates are NOT built do not always have protein sequences listed (so generally less findable from a protein sequence point of view). However, these CryoEM maps may provide insight -- in particular for protein systems where there is lots of flexibility / dynamics and/or few to no experimentally-determined coordinates available.

To find CryoEM Maps without coordinates please visit:
<br>
[Electron Microscopy Data Bank (EMDB)](https://www.ebi.ac.uk/emdb/)

In [ ]:
#@title 4.1. List PDB ids found *via* **PDBe-KB** but not *via* **3D Beacons**
#@markdown In this sub-section any experimentally-determined structures found *via* **PDBe-KB** but not found *via* **3D Beacons** will be reported.
#@markdown <br>
#@markdown <br>

#Run this code to display the widget
UniProt_ID = "" #@param {type:"string"}

#@markdown <br>
#@markdown Use filter button and imput the residue number to see if there is an experimental structure for a particular genetic variant.
#@markdown
#@markdown <br>
#@markdown <br>

diff_btwn_PDBeKB_3DBeacons(UniProt_ID)

Copyright 2024 EMBL - European Bioinformatics Institute

Licensed under the Apache License, Version 2.0 (the "License"); you may not use this file except in compliance with the License. You may obtain a copy of the License at http://www.apache.org/licenses/LICENSE-2.0

Unless required by applicable law or agreed to in writing, software distributed under the License is distributed on an "AS IS" BASIS, WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied. See the License for the specific language governing permissions and limitations under the License.